In [2]:
import matplotlib.pyplot as plt
import numpy as np
from joblib import Parallel, delayed

import acoutreams

k0s = 2 * np.pi * np.linspace(50000, 500000, 200) / 343
materials = [acoutreams.AcousticMaterial(1050 + 100j, 2350 - 300j), 
             acoutreams.AcousticMaterial(998, 1497)]
kzs = [0]
mmax = 10
radius = 0.005
cylinders = [acoutreams.AcousticTMatrixC.cylinder(kzs, mmax, k0, radius, materials) for k0 in k0s]

UnboundLocalError: cannot access local variable 'k_cyl' where it is not associated with a value